<a href="https://colab.research.google.com/github/Ananth09/Image-Classification-using-DL/blob/main/imgclassify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [ ]:
data_dir = '/content/drive/MyDrive/img_class/'

#parameters
batch_size = 17
image_size = (224, 224)
num_epochs = 31

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.15

)

In [ ]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 750 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 131 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.ResNet101V2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)


171317808/171317808 [==============================] - 6s 0us/step


In [ ]:

flatten_layer = Flatten()(base_model.layers[-1].output)
output_layer=Dense(1, activation = 'sigmoid')(flatten_layer)
model = Model(inputs=base_model.input, outputs=output_layer)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    verbose=1
)

Epoch 1/31
45/45 [==============================] - 288s 5s/step - loss: 0.3190 - accuracy: 0.8720 - val_loss: 0.8266 - val_accuracy: 0.8015
Epoch 2/31
45/45 [==============================] - 11s 253ms/step - loss: 0.0911 - accuracy: 0.9733 - val_loss: 0.2273 - val_accuracy: 0.9237
Epoch 3/31
45/45 [==============================] - 11s 253ms/step - loss: 0.0688 - accuracy: 0.9733 - val_loss: 0.1126 - val_accuracy: 0.9618
Epoch 4/31
45/45 [==============================] - 11s 253ms/step - loss: 0.0953 - accuracy: 0.9720 - val_loss: 0.1766 - val_accuracy: 0.9389
Epoch 5/31
45/45 [==============================] - 12s 255ms/step - loss: 0.0660 - accuracy: 0.9760 - val_loss: 0.3235 - val_accuracy: 0.9542
Epoch 6/31
45/45 [==============================] - 12s 254ms/step - loss: 0.0820 - accuracy: 0.9760 - val_loss: 0.1898 - val_accuracy: 0.9389
Epoch 7/31
45/45 [==============================] - 12s 255ms/step - loss: 0.0260 - accuracy: 0.9947 - val_loss: 0.0755 - val_accuracy: 0.9695
E

In [ ]:
# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(val_generator, verbose=1)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")


8/8 [==============================] - 2s 209ms/step - loss: 0.5162 - accuracy: 0.8473
Validation Loss: 0.5162
Validation Accuracy: 84.73%


In [ ]:
model.save('resnet101.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

custom_image_path = '/content/drive/MyDrive/test1/0005.jpg'
custom_image = load_img(custom_image_path, target_size=(224, 224))
custom_image = img_to_array(custom_image)
custom_image = custom_image / 255.0  # Normalize pixel values
custom_image = custom_image.reshape(1, 224, 224, 3)  # Reshape for model input


In [ ]:
# Make predictions
prediction = model.predict(custom_image)
print(prediction)
# Interpret the prediction
if prediction > 0.5:
    result = "lifestyle"
else:
    result = "plain_background"

print(f"The predicted class is: {result}")


1/1 [==============================] - 0s 29ms/step
[[1.]]
The predicted class is: lifestyle


In [ ]:
import os
from PIL import Image
folder_path = '/content/drive/MyDrive/test_for_wrong'


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
plain_background_paths=[]
liffes=[]
life=0
plain=0
for filename in os.listdir(folder_path):
  if filename.endswith('.jpg'):
    #if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(folder_path, filename)

        # Load and preprocess the image
        custom_image = load_img(image_path, target_size=(224, 224))
        custom_image = img_to_array(custom_image)
        custom_image = custom_image / 255.0
        custom_image = custom_image.reshape(1, 224, 224, 3)

        # Predict using your model
        prediction = model.predict(custom_image)
        print(prediction)
        # Interpret the prediction
        if prediction < 0.5:
            result = "plain_background"
            plain_background_paths.append(image_path)
            plain += 1
        else:
            result = "lifestyle"
            liffes.append(image_path)
            life += 1

print(f"Number of 'lifestyle' images: {life}")
print(f"Number of 'plain_background' images: {plain}")
print("Paths of 'plain_background' images:")
for path in plain_background_paths:
    print(path)
print("Paths of 'lifestyle' images:")

for path in liffes:
    print(path)

1/1 [==============================] - 0s 31ms/step
[[5.208779e-07]]
1/1 [==============================] - 0s 27ms/step
[[1.]]
1/1 [==============================] - 0s 26ms/step
[[0.9999888]]
1/1 [==============================] - 0s 27ms/step
[[1.]]
1/1 [==============================] - 0s 27ms/step
[[1.]]
1/1 [==============================] - 0s 27ms/step
[[1.]]
1/1 [==============================] - 0s 26ms/step
[[1.]]
1/1 [==============================] - 0s 28ms/step
[[1.]]
1/1 [==============================] - 0s 34ms/step
[[0.9999995]]
1/1 [==============================] - 0s 27ms/step
[[1.]]
1/1 [==============================] - 0s 27ms/step
[[1.]]
1/1 [==============================] - 0s 26ms/step
[[1.]]
1/1 [==============================] - 0s 27ms/step
[[0.9999995]]
1/1 [==============================] - 0s 27ms/step
[[1.]]
1/1 [==============================] - 0s 30ms/step
[[1.]]
1/1 [==============================] - 0s 27ms/step
[[1.]]
1/1 [====================